In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib

# Load your list of employee data from a CSV file
employee_data = pd.read_csv('user_data.csv')

# Load the trained model and thresholds
# Load the Random Forest classifier model using joblib
clf = joblib.load('trained_model.pkl')

# Define thresholds for different categories
senior_threshold = 0.7
junior_threshold = 0.3

# Initialize label encoder
label_encoder = LabelEncoder()

# Preprocess the employee data similar to the training data
employee_data['Education Level'] = label_encoder.fit_transform(employee_data['Education Level'])
employee_data['Gender'] = label_encoder.fit_transform(employee_data['Gender'])
employee_data['Current Position of the Job'] = label_encoder.fit_transform(employee_data['Current Position of the Job'])
employee_data['Polling Station'] = label_encoder.fit_transform(employee_data['Polling Station'])
employee_data['Experience'] = employee_data['Experience'].str.extract('(\d+)').astype(float)
employee_data['Current Salary'] = employee_data['Current Salary'].str.replace('[\$,]', '', regex=True).astype(float)

# Predict positions for each employee
predictions = []

for index, row in employee_data.iterrows():
    new_data = pd.DataFrame({
        'Age': [row['Age']],
        'Education Level': [row['Education Level']],
        'Gender': [row['Gender']],
        'Polling Station': [row['Polling Station']],
        'Current Position of the Job': [row['Current Position of the Job']],
        'Experience': [row['Experience']],
        'Current Salary': [row['Current Salary']]
    })

    new_prediction_probs = clf.predict_proba(new_data)
    
    if new_prediction_probs[0, 2] >= senior_threshold:
        new_prediction = "Senior"
    elif new_prediction_probs[0, 1] >= junior_threshold:
        new_prediction = "Junior"
    else:
        new_prediction = "Clerk"
    
    predictions.append(new_prediction)

# Add the predicted positions to the employee_data DataFrame
employee_data['Predicted Position'] = predictions

# Save the employee data with predicted positions to a CSV file
employee_data.to_csv('employee_data_with_predictions.csv', index=False)


